# Find All Words on a Board
Given a 2D board of characters and an array of words, find all the words in the array that can be formed by tracing a path through adjacent cells in the board. Adjacent cells are those which horizontally or vertically neighbor each other. We can't use the same cell more than once for a single word.

```python
Input: board = [['b', 'y', 's'], ['r', 't', 'e'], ['a', 'i', 'n']],
       words = ['byte', 'bytes', 'rat', 'rain', 'trait', 'train']
Output: ['byte', 'bytes', 'rain', 'train']
```

## Intuition
This problem has multiple layers, so let's start with a **simpler version**, where we only search for **one word** on the board.

---

## **Simplified Problem: Single Word Search**
If the **words array contains only one word**, a straightforward approach is:

1. Iterate through each **cell** of the board.
2. If a cell contains the **first letter** of the word, perform a **DFS** from that cell in all four directions (**left, right, up, down**) to find the rest of the word.
3. Use **backtracking**:  
   - If we cannot find the next letter in any adjacent cell, we backtrack.  
   - The process continues until we find the word or exhaust all possibilities.

---

## **Original Problem: Searching for Multiple Words**
The above approach works well for **one word**, but repeating this process for **every word** in the array is inefficient. Let's optimize the search.

### **Optimizing with a Trie**
Instead of restarting the search for each word, we can use a **trie**, which allows us to efficiently search words that share prefixes. This prevents redundant searches and speeds up word retrieval.

- **Trie Structure:**  
  - Each node represents a character in one or more words.
  - We store the **word attribute** at the end of a valid word to determine which specific word has been found.

Using this **trie**, we can now search for multiple words on the board efficiently by checking if any paths in the **trie** correspond to paths on the **board**.

---

## **Backtracking with a Trie**
### **Step 1: Finding a Starting Point**
- We iterate through the board until we find a **cell** that matches **any child node of the trie's root**, representing the first letter of a word.
- Example: If cell `(0,0)` contains **'b'**, and 'b' is a child of the **trie root**, we start **DFS** from this cell.

### **Step 2: Searching for Words**
- For each matching cell:
  - Check its **adjacent** cells.
  - If an adjacent cell contains a letter that corresponds to the next trie node, continue the **DFS**.
  - Example: If `'b'` is followed by `'y'`, and `'y'` is a child of `'b'` in the trie, we move to `'y'`.

- We **continue** this process until:
  - No more matching trie nodes are found.
  - We reach a **trie node that marks the end of a word**, at which point we record the word in our output.

### **Step 3: Handling Backtracking**
- To **prevent revisiting cells**, we track visited cells using:
  - A **hash set**.
  - **In-place marking**: Temporarily modifying the visited cell to a special character.
- Since we should **not permanently modify the board**, we must **restore** the original character at the end of each recursive call.

```python
def dfs(r, c, board, node):
    temp = board[r][c]
    board[r][c] = '#'
    for next_r, next_c in adjacent_cells:
        if board[next_r][next_c] in node.children:
            dfs(
                next_r,
                next_c,
                board,
                node.children[board[next_r][next_c]]
            )
    board[r][c] = temp
```

For each cell on the board that matches a character of one of the root node's children, make a recursive DFS call to that cell, passing in the corresponding node. At each of these DFS calls:
1. Check if the current node represents the end of a word. If it does, add that word to the output.
2. Mark the current cell as visited by setting the cell to `'#'`.
3. Recursively explore all adjacent cells that correspond with a child of the current TrieNode.
4. Backtrack by reverting the cell back to its original character.

---

### Handling Multiple Occurrences of the Same Word
Since the board may contain the same word in multiple locations, we need to avoid adding duplicate words to the output. To do this:
- Each TrieNode has a `word` attribute to indicate the end of a word.
- After recording a word in our output, we set that node's word attribute to `None`, ensuring we cannot record the same word again.

In [1]:
from typing import List

class TrieNode:
    def __init__(self):
        self.children = {}
        self.word = None

def find_all_words_on_a_board(board: List[List[str]], words: List[str]) -> List[str]:
    root = TrieNode()

    for word in words:
        node = root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.word = word
    
    res = []
    for r in range(len(board)):
        for c in range(len(board[0])):
            if board[r][c] in root.children:
                dfs(board, r, c, root.children[board[r][c]], res)
    return res

def dfs(board: List[List[str]], r: int, c: int, node: TrieNode, res: List[str]) -> None:

    if node.word:
        res.append(node.word)
        node.word = None

    temp = board[r][c]
    board[r][c] = '#'
    dirs = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    for d in dirs:
        next_r, next_c = r + d[0], c + d[1]
        if (is_within_bounds(next_r, next_c, board) and board[next_r][next_c] in node.children):
            dfs(
                board, next_r, next_c,
                node.children[board[next_r][next_c]],
                res
            )
    board[r][c] = temp

def is_within_bounds(r: int, c: int, board: List[str]) -> bool:
    return 0 <= r < len(board) and 0 <= c < len(board[0])

### Complexity Analysis

#### Time Complexity
The time complexity is **O(N * L + m * n * 3<sup>L</sup>)**, where:  
- **N** denotes the number of words in the `words` array.  
- **L** denotes the length of the longest word.  
- **m * n** denotes the size of the board.  

Breakdown:
- **Building the Trie:**  
  - We insert each word from the input array into the Trie, with each word containing a maximum of **L** characters.  
  - This takes **O(N * L)** time.  

- **Main Search Process (DFS Traversal):**  
  - We perform a **DFS** for each of the **m * n** cells on the board.  
  - Each DFS call takes **O(3<sup>L</sup>)** time because:  
    - At each point in the DFS, we make up to **3 recursive calls** (one for each of the three adjacent cells, excluding the cell we came from).  
  - This is repeated for at most the length of the longest word, **L**.  

Thus, the final time complexity is **O(N * L + m * n * 3<sup>L</sup>)**.

---

#### Space Complexity
The space complexity is **O(N * L)**.

Breakdown:
- **Trie Storage:**  
  - The Trie has a space complexity of **O(N * L)**.  
  - In the worst case, if all words have unique prefixes, we store every character of every word in the Trie.  
  - Each word attribute stored at the end of a path in the Trie takes **O(L)** space.  
  - With **N** words, this contributes an additional **O(N * L)** space.  

- **Recursive Call Stack:**  
  - The maximum depth of the recursive call stack is **L**, leading to an additional **O(L)** space in the worst case.

Thus, the overall space complexity is **O(N * L)**.